In [1]:
!python scripts/modify_configs.py --path /home/thinh_do/Workplace/ResGCNv1/resources  --ntu60_path /home/thinh_do/Workplace/ResGCNv1/NTU_RGBD/nturgbd_skeletons_s001_to_s017  --ntu120_path /home/thinh_do/Workplace/ResGCNv1/NTU_RGBD/nturgbd_skeletons_s018_to_s032


In [2]:
!bash scripts/auto_gen_data.sh

[ 2022-04-13 23:24:58,644 ] 
[ 2022-04-13 23:24:58,644 ] Starting generating ...
[ 2022-04-13 23:24:58,644 ] Dataset: ntu-xsub
[ 2022-04-13 23:24:58,644 ] Phase: train
[ 2022-04-13 23:24:59,706 ] Phase: eval
[ 2022-04-13 23:25:00,878 ] Finish generating!
[ 2022-04-13 23:25:04,364 ] 
[ 2022-04-13 23:25:04,364 ] Starting generating ...
[ 2022-04-13 23:25:04,364 ] Dataset: ntu-xview
[ 2022-04-13 23:25:04,364 ] Phase: train
[ 2022-04-13 23:25:05,366 ] Phase: eval
[ 2022-04-13 23:25:06,547 ] Finish generating!
[ 2022-04-13 23:25:10,019 ] 
[ 2022-04-13 23:25:10,019 ] Starting generating ...
[ 2022-04-13 23:25:10,020 ] Dataset: ntu-xsub120
[ 2022-04-13 23:25:10,020 ] Phase: train
[ 2022-04-13 23:25:11,776 ] Phase: eval
[ 2022-04-13 23:25:13,769 ] Finish generating!
[ 2022-04-13 23:25:16,486 ] 
[ 2022-04-13 23:25:16,486 ] Starting generating ...
[ 2022-04-13 23:25:16,486 ] Dataset: ntu-xset120
[ 2022-04-13 23:25:16,486 ] Phase: train
[ 2022-04-13 23:25:18,253 ] Phase: eval
[ 2022-04-13 23:25:1

In [3]:

!python main.py --config 1008 -v -e -vc 30 -pp /home/thinh-do/Workspace/ResGCNv1/pretrained/

[ 2022-04-13 23:25:42,628 ] 
[ 2022-04-13 23:25:42,628 ] Starting generating ...
[ 2022-04-13 23:25:42,628 ] Dataset: ntu-xset120
[ 2022-04-13 23:25:42,628 ] Phase: train
[ 2022-04-13 23:25:43,742 ] Phase: eval
[ 2022-04-13 23:25:44,919 ] Finish generating!


In [11]:
from src.utils import *
import os,time
import logging

In [16]:

path='/home/thinh_do/Workplace/ResGCNv1/pretrained'
model=load_checkpoint1(path,model_name='1008_resgcn-b19_ntu-xset120.pth.tar')

In [17]:
model

In [ ]:
import os,cv2
import numpy as np
from time import strftime
import logging

import threading

import mediapipe as mp
import time

from torch import imag
# from clearml import Task,Logger
# from tensorboardX import SummaryWriter
from Utils import make_landmark_timestep,init_parameters,detect,read_data,init_model,draw_class_on_image

import imutils
from imutils.video import FPS

# task = Task.init(project_name='Action Recognition', task_name='task_1')
# S_writer=SummaryWriter('run/Action Recognition')

def main():
    print ('--------------------------------')
    parser = init_parameters()
    args, _ = parser.parse_known_args()

    net=init_model(args)
    print(net)
    label="NULL..."
    mpPose = mp.solutions.pose
    pose = mpPose.Pose()
    mpDraw = mp.solutions.drawing_utils
    mp_drawing_styles = mp.solutions.drawing_styles
    path_video='/home/thinh_do/Workplace/ResGCNv1/1a.mp4'
    print("================= Start detect ===============")
    video=cv2.VideoCapture(path_video)
    time.sleep(1.0)
    fps=FPS().start()
    
    while video.isOpened():
        success,image=video.read()
        imageRGB=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = pose.process(imageRGB)
        
        
        mpDraw.draw_landmarks(
            image,
            results.pose_landmarks,
            mpPose.POSE_CONNECTIONS,
            landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())
        

        image = draw_class_on_image(label, image)


        fps.update()
        fps.stop()
        cv2.putText(image, "FPS: {}".format(int(fps.fps())), (10,70), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0), 2)
        cv2.imshow("Changed", image)
        if cv2.waitKey(5) & 0xFF == ord('q'):  # press q to quit
            break
        
    cv2.destroyAllWindows()
        
if __name__ == '__main__':
    main()
    

In [2]:
import cv2
video = cv2.VideoCapture('/home/thinh_do/Workplace/ResGCNv1/video.mp4')
while video.isOpened():
    ss,image=video.read()
    print(image)
    cv2.imshow("Changed", image)
    if cv2.waitKey(5) & 0xFF == ord('q'):  # press q to quit
        break
        
cv2.destroyAllWindows()

In [ ]:
def main():
    args = parse_args()

    frame_paths, original_frames = frame_extraction(args.video)
    num_frame = len(frame_paths)
    h, w, _ = original_frames[0].shape

    # Get Human detection results and pose results
    human_detections = detection_inference(args, frame_paths)
    pose_results = None
    if args.use_skeleton_recog or args.use_skeleton_stdet:
        pose_results = pose_inference(args, frame_paths, human_detections)

    # resize frames to shortside 256
    new_w, new_h = mmcv.rescale_size((w, h), (256, np.Inf))
    frames = [mmcv.imresize(img, (new_w, new_h)) for img in original_frames]
    w_ratio, h_ratio = new_w / w, new_h / h

    # Load spatio-temporal detection label_map
    stdet_label_map = load_label_map(args.label_map_stdet)
    rgb_stdet_config = mmcv.Config.fromfile(args.rgb_stdet_config)
    rgb_stdet_config.merge_from_dict(args.cfg_options)
    try:
        if rgb_stdet_config['data']['train']['custom_classes'] is not None:
            stdet_label_map = {
                id + 1: stdet_label_map[cls]
                for id, cls in enumerate(rgb_stdet_config['data']['train']
                                         ['custom_classes'])
            }
    except KeyError:
        pass

    action_result = None
    if args.use_skeleton_recog:
        print('Use skeleton-based recognition')
        action_result = skeleton_based_action_recognition(
            args, pose_results, num_frame, h, w)
    else:
        print('Use rgb-based recognition')
        action_result = rgb_based_action_recognition(args)

    stdet_preds = None
    if args.use_skeleton_stdet:
        print('Use skeleton-based SpatioTemporal Action Detection')
        clip_len, frame_interval = 30, 1
        timestamps, stdet_preds = skeleton_based_stdet(args, stdet_label_map,
                                                       human_detections,
                                                       pose_results, num_frame,
                                                       clip_len,
                                                       frame_interval, h, w)
        for i in range(len(human_detections)):
            det = human_detections[i]
            det[:, 0:4:2] *= w_ratio
            det[:, 1:4:2] *= h_ratio
            human_detections[i] = torch.from_numpy(det[:, :4]).to(args.device)

    else:
        print('Use rgb-based SpatioTemporal Action Detection')
        for i in range(len(human_detections)):
            det = human_detections[i]
            det[:, 0:4:2] *= w_ratio
            det[:, 1:4:2] *= h_ratio
            human_detections[i] = torch.from_numpy(det[:, :4]).to(args.device)
        timestamps, stdet_preds = rgb_based_stdet(args, frames,
                                                  stdet_label_map,
                                                  human_detections, w, h,
                                                  new_w, new_h, w_ratio,
                                                  h_ratio)

    stdet_results = []
    for timestamp, prediction in zip(timestamps, stdet_preds):
        human_detection = human_detections[timestamp - 1]
        stdet_results.append(
            pack_result(human_detection, prediction, new_h, new_w))

    def dense_timestamps(timestamps, n):
        """Make it nx frames."""
        old_frame_interval = (timestamps[1] - timestamps[0])
        start = timestamps[0] - old_frame_interval / n * (n - 1) / 2
        new_frame_inds = np.arange(
            len(timestamps) * n) * old_frame_interval / n + start
        return new_frame_inds.astype(np.int)

    dense_n = int(args.predict_stepsize / args.output_stepsize)
    output_timestamps = dense_timestamps(timestamps, dense_n)
    frames = [
        cv2.imread(frame_paths[timestamp - 1])
        for timestamp in output_timestamps
    ]

    print('Performing visualization')
    pose_model = init_pose_model(args.pose_config, args.pose_checkpoint,
                                 args.device)

    if args.use_skeleton_recog or args.use_skeleton_stdet:
        pose_results = [
            pose_results[timestamp - 1] for timestamp in output_timestamps
        ]

    vis_frames = visualize(frames, stdet_results, pose_results, action_result,
                           pose_model)
    vid = mpy.ImageSequenceClip([x[:, :, ::-1] for x in vis_frames],
                                fps=args.output_fps)
    vid.write_videofile(args.out_filename)

    tmp_frame_dir = osp.dirname(frame_paths[0])
    shutil.rmtree(tmp_frame_dir)


In [ ]:
ROOT = os.path.dirname(os.path.abspath(__file__))+"/"
    sys.path.append(ROOT)
    cfg_all = read_yaml(ROOT +"class.yaml")
    CLASSES = np.array(cfg_all["classes"])